In [7]:
import sagemaker, boto3
from sagemaker import get_execution_role
from sagemaker import image_uris
from sagemaker.sklearn import SKLearn

instance_type = 'ml.m5.12xlarge' #ml.g4dn.xlarge(T4) #ml.m5.4xlarge(16), ml.m5.12xlarge (48) ml.m5.24xlarge (96)
project_bucket = 'rickmartelflood'
model_folder = 'model'
model_output = 's3://{}/{}'.format(project_bucket, model_folder)
print(f'Model output bucket: {model_output}')

Model output bucket: s3://rickmartelflood/model


In [10]:
%%time
aws_role = get_execution_role()
sagemaker_session = sagemaker.Session()

image_uri = image_uris.retrieve(framework='sklearn', region='us-east-1',
                    version='1.2-1', py_version='py3',
                    image_scope='training',
                    instance_type=instance_type)

env = {'SAGEMAKER_REQUIREMENTS': 'requirements.txt'}

model = SKLearn(
    role=aws_role,
    sagemaker_session=sagemaker_session,
    output_path=model_output,
    code_location=model_output,
    entry_point="hpo_ex1_case1.py",
    source_dir='./container_scripts',
    env=env,
    image_uri=image_uri,
    instance_count=1,
    instance_type=instance_type,
    hyperparameters={'n_trials': 20, 
                     'n_study_jobs': 5, 'n_model_jobs': 9,
                     'training_fraction': 1.0},
    use_spot_instances=True,
    max_run=600*15, 
    max_wait=600*15,
)

model.fit()

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-05-26-22-11-50-407


2024-05-26 22:11:50 Starting - Starting the training job...
2024-05-26 22:12:05 Starting - Preparing the instances for training...
2024-05-26 22:12:43 Downloading - Downloading the training image...
2024-05-26 22:13:08 Training - Training image download completed. Training in progress.....2024-05-26 22:13:47,613 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-05-26 22:13:47,615 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-26 22:13:47,618 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-05-26 22:13:47,632 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-05-26 22:13:47,882 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━